In [ ]:
import os
import json
import time
import uuid          # ← Added for long clientToken
import boto3

REGION = os.getenv("REGION", "us-east-1")
SYNC_TARGETS = os.getenv("SYNC_TARGETS", "")  # Format: KB1:DS1,KB2:DS2,...

bedrock_agents = boto3.client("bedrock-agent", region_name=REGION)


def parse_sync_targets(raw: str):
    pairs = []
    for entry in raw.split(","):
        entry = entry.strip()
        if not entry:
            continue
        try:
            kb, ds = entry.split(":")
            pairs.append((kb.strip(), ds.strip()))
        except ValueError:
            print(f"Invalid entry in SYNC_TARGETS: {entry}")
    return pairs


def wait_for_completion(kb_id, ds_id, job_id, timeout=600, poll=10):
    """
    Polls every `poll` seconds until ingestion job finishes or timeout.
    """
    start = time.time()

    while time.time() - start < timeout:
        resp = bedrock_agents.get_ingestion_job(
            knowledgeBaseId=kb_id,
            dataSourceId=ds_id,
            ingestionJobId=job_id
        )

        status = resp["ingestionJob"]["status"]
        print(f"[{kb_id}] Job {job_id} status: {status}")

        if status in ("COMPLETE", "FAILED", "STOPPED"):
            return status
        
        time.sleep(poll)

    return "TIMED_OUT"


def lambda_handler(event, context):

    targets = parse_sync_targets(SYNC_TARGETS)
    if not targets:
        raise ValueError(
            "Set SYNC_TARGETS env var like KB1:DS1,KB2:DS2"
        )

    final_results = []

    for kb_id, ds_id in targets:

        print(f"🔄 Starting ingestion for KB={kb_id}, DS={ds_id}")
        
        # ------- FIXED: clientToken must be >= 33 chars -------
        client_token = f"sync-{kb_id}-{int(time.time())}-{uuid.uuid4().hex}"
        
        start_resp = bedrock_agents.start_ingestion_job(
            knowledgeBaseId=kb_id,
            dataSourceId=ds_id,
            clientToken=client_token,
            description="Scheduled sync via Lambda",
        )

        job = start_resp["ingestionJob"]
        job_id = job["ingestionJobId"]

        print(f"🚀 Ingestion started | Job ID: {job_id}")

        # ---- WAIT FOR FINAL STATUS ----
        final_status = wait_for_completion(kb_id, ds_id, job_id)

        result = {
            "knowledgeBaseId": kb_id,
            "dataSourceId": ds_id,
            "ingestionJobId": job_id,
            "finalStatus": final_status,
        }

        print(f"✅ Final result: {json.dumps(result)}")
        final_results.append(result)

    return {
        "statusCode": 200,
        "body": json.dumps({"results": final_results}),
    }
